In [1]:
import numpy as np
import rasterio
import shapely
from czml3 import Document, Preamble
from czml3.properties import Color, Material, Polygon, Polyline, SolidColorMaterial

import czml3_ext
from czml3_ext import packets
from czml3_ext.colours import RGBA, create_palette
from czml3_ext.helpers import get_border, get_contours

In [ ]:
print(f"Using czml3-ext version: {czml3_ext.__version__}")

## Border
A single border

In [3]:
border = packets.border("Israel")

Multiple borders

In [4]:
borders = packets.border(["syria", "lebanon", "jordan"])

## Sensors
A single sensor

In [5]:
sensor1 = packets.sensor_polyline(
    np.array([[31.4], [34.7], [0]]),
    10,
    10,
    30,
    20,
    10_000,
    5000,
    name="A Sensor",  # kwargs add to the CZML3 packet
)

Multiple sensors

In [6]:
ddm_LLA = np.array([[[31.75], [34.72], [0]], [[31.8], [34.68], [0]]])
sensors1 = packets.sensor_polyline(
    ddm_LLA,
    [50, 90],
    [60, 30],
    [10, 50],
    [20, 20],
    [10_000, 20_000],
    [5_000, 2_000],
    name=[
        f"Sensor #{i+1}" for i in range(ddm_LLA.shape[0])
    ],  # a kwarg sequence assigns each property to their appropriate sensor
    polyline=Polyline(
        positions=[],  # required to create a Polyline(), and is ignored by czml3-ext
        material=Material(solidColor=SolidColorMaterial(color=Color(rgba=RGBA.blue))),
    ),  # a kwarg that is not a sequence will be applied to all sensors
)

In [7]:
sensors2 = packets.sensor_polygon(
    np.array([[[31.75], [34.72], [0]], [[31.8], [34.68], [0]]]),
    [50, 90],
    [60, 30],
    [10, 50],
    [20, 20],
    [10_000, 20_000],
    [5_000, 2_000],
    name=[f"Sensor #{i+1}" for i in range(ddm_LLA.shape[0])],
    polygon=Polygon(
        positions=[],
        material=Material(solidColor=SolidColorMaterial(color=Color(rgba=RGBA.blue))),
    ),
)

## Grid
Create a grid in Israel

In [8]:
ddm_LLA_israel = get_border("israel")
poly_shape = shapely.Polygon(ddm_LLA_israel[:, [0, 1], 0].tolist())

In [9]:
x = np.linspace(ddm_LLA_israel[:, 1, 0].min(), ddm_LLA_israel[:, 1, 0].max(), 20)
y = np.linspace(ddm_LLA_israel[:, 0, 0].min(), ddm_LLA_israel[:, 0, 0].max(), 20)
xv, yv = np.meshgrid(x, y)
ddm_points = []
for i in range(x.shape[0]):
    for j in range(y.shape[0]):
        if not poly_shape.contains(shapely.Point(yv[j, i], xv[j, i])):
            continue
        ddm_points.append([yv[j, i], xv[j, i], 0])
ddm_LLA_points = np.array(ddm_points).reshape((-1, 3, 1))

Define the colour palette

In [10]:
rgba = create_palette([RGBA.blue, RGBA.white, RGBA.blue], ddm_LLA_points.shape[0])

Create the grid

In [11]:
grid = packets.grid(
    ddm_LLA_points,
    polygon=[
        Polygon(
            positions=[],
            material=Material(solidColor=SolidColorMaterial(color=Color(rgba=c))),
        )
        for c in rgba
    ],
    name=[f"Square {i}" for i in range(len(rgba))],
    description=[
        f"Colour: [{c[0]:.2f}, {c[1]:.2f}, {c[2]:.2f}, {c[3]:.2f}]" for c in rgba
    ],
)

## Coverage

In [12]:
with rasterio.open("data.tif") as src:
    arr = src.read(1)
    origin_x, origin_y = src.transform * (0, 0)
    delta_x = src.transform[0]
    delta_y = src.transform[4]

In [13]:
coverages, holes = get_contours(
    arr == 5,
    origin_x,
    delta_x,
    origin_y,
    delta_y,
)

In [14]:
vs = packets.coverage(
    coverages,
    holes,
    polygon=Polygon(
        positions=[],
        material=Material(solidColor=SolidColorMaterial(color=Color(rgba=RGBA.black))),
    ),
)

In [15]:
sensor_coverage = packets.sensor_polyline(
    np.array(
        [
            [origin_y + arr.shape[0] / 2 * delta_y],
            [origin_x + arr.shape[1] / 2 * delta_x],
            [0],
        ]
    ),
    320,
    20,
    180,
    20,
    10_000,
)

## Export
Export to file as you would any other `czml3.Document`.

In [16]:
doc = Document(
    [Preamble(name="simple")]
    + border
    + borders
    + sensor1
    + sensors1
    + sensors2
    + grid
    + vs
    + sensor_coverage
)

In [17]:
with open("example.czml", "w") as f:
    doc.dump(f)